# Deploying Model

In [ ]:
#!pip3 install pydantic[dotenv]

In [ ]:
#!pip3 install arch --upgrade

In [ ]:
#pip install os

In [ ]:
#!pip3 install arch --user

In [31]:

import flask


In [20]:

#import os
import sqlite3
from glob import glob

import joblib
import pandas as pd
import requests
from arch import arch_model
from statsmodels.tsa.arima_model import ARIMA
from arch.univariate.base import ARCHModelResult
from data import SQLRepository, AlphaVantageAPI
from config import settings


In [2]:
# Connection to databse
connection = sqlite3.connect(settings.db_name, check_same_thread=False)
repo =SQLRepository(connection=connection)

print("repo type:", type(repo))
print("repo.connection type:", type(repo.connection))

repo type: <class 'data.SQLRepository'>
repo.connection type: <class 'sqlite3.Connection'>


In [3]:
os.getcwd()

<IPython.core.display.Javascript object>

'C:\\Users\\ITC_Consultant\\Documents\\Personal Studies\\Deployment in flask api'

In [4]:
# Instantiate the garch model class

from model import GarchModel
from_curr='GBP'
to_curr='USD'
# Instantiate a `GarchModel`
curr_data_empty= GarchModel(from_curr=from_curr,to_curr=to_curr, repo=repo, use_new_data=False)

# Does `gm_gbp` have the correct attributes?
assert curr_data_empty.from_currency == from_curr
assert curr_data_empty.to_currency == to_curr
assert curr_data_empty.repo == repo
assert not curr_data_empty.use_new_data
assert curr_data_empty.model_directory == settings.model_directory


In [7]:
# Instantiate `GarchModel`, use new data
currency_data = GarchModel(from_curr=from_curr,to_curr=to_curr, repo=repo, use_new_data=True)

# Check that model doesn't have `data` attribute yet
assert not hasattr(currency_data, "data")

# Wrangle data
currency_data.wrangle_data(n_observations=1000)

# Does model now have `data` attribute?
assert hasattr(currency_data, "data")
# does the model has repo connection
assert isinstance(currency_data.repo, SQLRepository)


# Is the `data` a Series?
assert isinstance(currency_data.data, pd.Series)

# Is Series correct shape?
assert currency_data.data.shape == (1000,)

currency_data.data.tail()

date
2023-04-24    0.375483
2023-04-25   -0.600769
2023-04-26    0.467403
2023-04-27    0.210155
2023-04-28   -0.224922
Name: return, dtype: float64

In [ ]:
#original with garch

In [11]:
# Instantiate `GarchModel`, use old data
currency_model = GarchModel(from_curr=from_curr,to_curr=to_curr, repo=repo, use_new_data=True)

# Wrangle data
currency_model.wrangle_data(n_observations=1000)

# Fit GARCH(1,1) model to data
currency_model.fit(p=1, q=1)

# Does `model_shop` have a `model` attribute now?
assert hasattr(currency_model, "model")

# Is model correct data type?
assert isinstance(currency_model.model, ARCHModelResult)

# Does model have correct parameters?
assert currency_model.model.params.index.tolist() == ["mu", "omega", "alpha[1]", "beta[1]"]

# Check model parameters
currency_model.model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                     Constant Mean - GARCH Model Results                      
==============================================================================
Dep. Variable:                 return   R-squared:                       0.000
Mean Model:             Constant Mean   Adj. R-squared:                  0.000
Vol Model:                      GARCH   Log-Likelihood:               -856.438
Distribution:                  Normal   AIC:                           1720.88
Method:            Maximum Likelihood   BIC:                           1740.51
                                        No. Observations:                 1000
Date:                Fri, Apr 28 2023   Df Residuals:                      999
Time:                        15:23:41   Df Model:                            1
                                  Mean Model                                 
=============================================================================
                 coef    std err          t      P>|t|       95.0% Conf. Int.
-----------------------------------------------------------------------------
mu         6.1564e-03  1.651e-02      0.373      0.709 [-2.621e-02,3.852e-02]
                              Volatility Model                              
============================================================================
                 coef    std err          t      P>|t|      95.0% Conf. Int.
----------------------------------------------------------------------------
omega          0.0243  1.050e-02      2.316  2.058e-02 [3.736e-03,4.490e-02]
alpha[1]       0.1371  3.608e-02      3.801  1.441e-04   [6.642e-02,  0.208]
beta[1]        0.7998  5.522e-02     14.484  1.526e-47     [  0.692,  0.908]
============================================================================

Covariance estimator: robust
"""

In [17]:
print(dir(currency_model))
currency_model.__getattribute__

['_GarchModel__clean_prediction', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', 'aic', 'bic', 'data', 'dump', 'fit', 'from_currency', 'load', 'model', 'model_directory', 'predict_volatility', 'repo', 'to_currency', 'use_new_data', 'wrangle_data']


<method-wrapper '__getattribute__' of GarchModel object at 0x000001C1307AF880>

In [10]:
# Generate prediction from `model_shop`
prediction = currency_model.predict_volatility(horizon=5)


# Is prediction a dictionary?
assert isinstance(prediction, dict)

# Are keys correct data type?
assert all(isinstance(k, str) for k in prediction.keys())

# Are values correct data type?
assert all(isinstance(v, float) for v in prediction.values())

prediction

{'2023-05-01T00:00:00': 0.460438504928745,
 '2023-05-02T00:00:00': 0.4721415625746844,
 '2023-05-03T00:00:00': 0.48284934329872264,
 '2023-05-04T00:00:00': 0.49267079527665963,
 '2023-05-05T00:00:00': 0.501698502421018}

In [ ]:
# just rough work

In [19]:
ticker=from_curr+to_curr
timestamp = pd.Timestamp.now().isoformat()
timestamp
p='models'

os.path.join(p+'/'+f'{timestamp}_{ticker}.pkl')


<IPython.core.display.Javascript object>

'models/2023-04-28T15:31:40.864153_GBPUSD.pkl'

In [ ]:
# Save `model_shop` model, assign filename
filename = model_shop.dump()

# Is `filename` a string?
assert isinstance(filename, str)

# Does filename include ticker symbol?
assert model_shop.ticker in filename

# Does file exist?
assert os.path.exists(filename)

filename

In [ ]:
# rough work
ticker='GBP'
#ticker='AMBUJACEM.BSE'
pattern=os.path.join(settings.model_directory, f'*{ticker}.pkl')
pattern
model_path=sorted(glob(pattern))[-1]
model_path

In [ ]:
def load(ticker):

    """Load latest model from model directory.

    Parameters  
    ----------
    ticker : str
        Ticker symbol for which model was trained.

    Returns
    -------
    `ARCHModelResult`
    """
    # Create pattern for glob search
    pattern = os.path.join(settings.model_directory, f'*{ticker}.pkl')

    # Try to find path of latest model
    try:
          
        model_path = sorted(glob(pattern))[-1]
    except IndexError:
        
        raise Exception(f'No model trained for "{ticker}".')
        
    
        

    # Handle possible `IndexError`
    

    # Load model
    model = joblib.load(model_path)

    # Return model
    return model

In [ ]:
model_shop

In [ ]:
# Assign load output to `model`
model_shop = load(ticker="GBP")

# Does function return an `ARCHModelResult`
assert isinstance(model_shop, ARCHModelResult)

# Check model parameters
model_shop.summary()

In [ ]:
model_shop = GarchModel(ticker="GBP", repo=repo, use_new_data=False)

# Check that new `model_shop_test` doesn't have model attached
assert not hasattr(model_shop, "model")

# Load model
model_shop.load()

# Does `model_shop_test` have model attached?
assert hasattr(model_shop, "model")

model_shop.model.summary()

Go to the command line, navigate to the directory for this project, and start your app server by entering the following command.


```bash
uvicorn main:app --reload --workers 1 --host localhost --port 8008
```
Remember how the AlphaVantage API had a "/query" path that we accessed using a get HTTP request? We're going to build similar paths for our application. Let's start with an MVP example so we can learn how paths work in FastAPI.

In [ ]:
#!pip3 install uvicorn

In [ ]:
#pip3 install fastapi

In [ ]:
#pip install uvicorn

In [ ]:
url = 'http://localhost:8008/hello'
response = requests.get(url=url)

print("response code:", response.status_code)
response.json()

In [ ]:
from main import FitIn, FitOut

# Instantiate `FitIn`. Play with parameters.
fi = FitIn(
    ticker='GBP',
    use_new_data=True,
    n_observations=2000,
    p=1,
    q=1
           
)
print(fi)

# Instantiate `FitOut`. Play with parameters.
fo =FitOut(
    ticker='GBP',
    use_new_data=True,
    n_observations=2000,
    p=1,
    q=1,
    success=True,
    message='Model is ready to rock'
    

)
print(fo)

In [ ]:
from main import build_model

# Instantiate `GarchModel` with function
model_shop = build_model(ticker="GBP", use_new_data=False)

# Is `SQLRepository` attached to `model_shop`?
assert isinstance(model_shop.repo, SQLRepository)

# Is SQLite database attached to `SQLRepository`
assert isinstance(model_shop.repo.connection, sqlite3.Connection)

# Is `ticker` attribute correct?
assert model_shop.ticker == "GBP"

# Is `use_new_data` attribute correct?
assert not model_shop.use_new_data



In [ ]:
type(model_shop.repo.connection)

In [ ]:
# URL of `/fit` path
url = 'http://localhost:8008/fit'

# Data to send to path
json = {
    "ticker":'GBP',
    "use_new_data":True,
    "n_observations":2000,
    "p":1,
    "q":1  
    }
# Response of post request
response =requests.post(url=url, json=json)
# Inspect response
print("response code:", response.status_code)
response.json()

In [ ]:
from main import PredictIn, PredictOut

pi = PredictIn(ticker="GBP", n_days=5)
print(pi)

po = PredictOut(
    ticker="GBP", n_days=5, success=True, forecast={}, message="success"
)
print(po)

In [ ]:
# URL of `/predict` path
url ="http://localhost:8008/predict"
# Data to send to path
json = {'ticker':'GBP', 'n_days':5}
# Response of post request
response = requests.post(url=url, json=json)
# Response JSON to be submitted to grader
submission = response.json()
# Inspect JSON
submission

In [ ]:
os.getcwd()

In [ ]:
model=joblib.load('models/2023-04-26T13_26_00.472128_GBP.pkl')

In [ ]:
prediction=model.forecast(horizon=3, reindex=False).variance
prediction

In [ ]:
model.summary()

In [ ]:
prediction.index[0]

In [ ]:
start_date=prediction.index[0] + pd.DateOffset(days=1)
start_date

In [ ]:
prediction_dates = pd.bdate_range(start=start_date, periods=prediction.shape[1], freq='B')
prediction_dates

In [ ]:
prediction_index=[d for d in prediction_dates.strftime('%d/%m/%Y')]
prediction_index

In [ ]:
data=prediction.values.flatten()**0.5
data
for i,j in zip(prediction_index, data):
    print(round(j,2))

In [ ]:
df=pd.DataFrame(data, index=prediction_index)
df.reset_index(inplace=True)

df.columns=['Date','Volatility']
# df.to_html()
df

'<table border="1" class="dataframe">\n  
    <thead>\n    
        <tr style="text-align: right;">\n      
            <th></th>\n      <th>0</th>\n    
        </tr>\n  </thead>\n  
        <tbody>\n    <tr>\n      <th>27/04/2023</th>\n      <td>0.493005</td>\n    </tr>\n    <tr>\n      <th>28/04/2023</th>\n      <td>0.502035</td>\n    </tr>\n    <tr>\n      <th>01/05/2023</th>\n      <td>0.510354</td>\n    </tr>\n  </tbody>\n</table>'

In [ ]:
prediction_listdate=[d for d in prediction_dates.strftime('%d/%m/%Y')]
prediction_listdate

In [ ]:
listdate=[]
for i in range(len(prediction_listdate)):
    p=prediction_listdate[0][:10]
    listdate.append(p)

In [ ]:
listdate

In [21]:
cur=AlphaVantageAPI()

In [24]:
df=cur.fx_daily(from_symbol='GBP', to_symbol='KES')

In [26]:
df.head().to_html()

'<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>open</th>\n      <th>high</th>\n      <th>low</th>\n      <th>close</th>\n    </tr>\n    <tr>\n      <th>date</th>\n      <th></th>\n      <th></th>\n      <th></th>\n      <th></th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>2023-04-28</th>\n      <td>169.35</td>\n      <td>170.64</td>\n      <td>169.06</td>\n      <td>170.45</td>\n    </tr>\n    <tr>\n      <th>2023-04-27</th>\n      <td>168.97</td>\n      <td>169.40</td>\n      <td>168.38</td>\n      <td>169.38</td>\n    </tr>\n    <tr>\n      <th>2023-04-26</th>\n      <td>168.00</td>\n      <td>169.47</td>\n      <td>167.93</td>\n      <td>168.89</td>\n    </tr>\n    <tr>\n      <th>2023-04-25</th>\n      <td>168.71</td>\n      <td>168.84</td>\n      <td>167.60</td>\n      <td>167.91</td>\n    </tr>\n    <tr>\n      <th>2023-04-24</th>\n      <td>167.84</td>\n      <td>168.65</td>\n      <td>167.51</td>\n  

In [27]:
df.close.values

array([170.45, 169.38, 168.89, ..., 141.79, 141.42, 141.23])

In [28]:
int(False)

0

In [30]:
bool('False')

True